### Importing libraries

In [1]:
import pandas as pd
from pathlib import Path
from datetime import date
from provenance_data_extract_package import prov_packages
package = prov_packages()

##### - Hashout the below line if you would like to extract the provenance data from a Standalone file present in the Input_files folder

In [3]:
# df_standalone_file = package.standalone_file_data_extract()
# df_standalone_file.to_excel("Summary_Report/StandaloneFile_Report.xlsx",index=False,sheet_name="Summary")

### Execution

##### Select the Testset and the comparator bucket

In [ ]:
Testset_BUCKET_NAME = package.select_bucket(description='Testset')
Comparator_BUCKET_NAME = package.select_bucket(description='Comparator')

##### Select the testruns for Testset and comparator

In [ ]:
testset = package.response_sub_bucket(bucket_name=Testset_BUCKET_NAME.value,title="Testset")
comparator = package.response_sub_bucket(bucket_name=Comparator_BUCKET_NAME.value,title="Comparator")

In [5]:
test_int_filename = package.sub_bucket_folders(BUCKET_NAME=Testset_BUCKET_NAME.value,Prefix=testset.value)
compar_int_filename = package.sub_bucket_folders(BUCKET_NAME=Comparator_BUCKET_NAME.value,Prefix=comparator.value)
testset_prefix_name = f"{testset.value}{test_int_filename}/"
comparator_prefix_name = f"{comparator.value}{compar_int_filename}/"
try:     
    testset_files = package.get_filenames(BUCKET_NAME=Testset_BUCKET_NAME.value,prefix_name=testset_prefix_name)
    comparator_files = package.get_filenames(BUCKET_NAME=Comparator_BUCKET_NAME.value,prefix_name=comparator_prefix_name)
    if len(comparator_files)==len(testset_files):
        dict_services = package.extract_data(testset_files=testset_files,comparator_files=comparator_files,Testset_BUCKET_NAME=Testset_BUCKET_NAME.value,Comparator_BUCKET_NAME=Comparator_BUCKET_NAME.value)
    else:
        dict_services = None
except Exception as e:
     print(str(e))

### Exporting summary report

In [6]:
if dict_services==None:
    print('No data extracted!!')
else:
    path = Path("Summary_Report")
    path.mkdir(parents=True,exist_ok=True)
    gitkeep_file = path / ".gitkeep"
    with open(gitkeep_file, 'w') as f:
        f.write("dummy file")
    comp_test = str(testset_files[0].split('/')[-3].split('+')[1])+"--"+str(comparator_files[0].split('/')[-3].split('+')[1])
    file_name = f"ProvSummary_{Testset_BUCKET_NAME.value}__{Comparator_BUCKET_NAME.value}_{comp_test}_{date.today()}"  
    df_out = pd.DataFrame()
    for sheet in list(dict_services.keys()):
        df_add = dict_services[sheet]
        df_out = pd.concat([df_out,df_add],ignore_index=True).reset_index(drop=True)

    df_final = df_out.style.map(package.highlight_true, subset=df_out.columns[-2:])
    df_final.to_excel(f"{path}/{file_name}.xlsx",index=False,sheet_name="Summary_Report")
    